# Data Mining in Action 2016 (осенний семестр)

## Задача на семинар: цветовая сегментация кожи на изображении

В этом примере мы построим простой классификатор с использованием библиотеки scikit-learn. В качестве данных мы возьмем выборку из UCI Репозитория (www.archive.ics.uci.edu), связанную с задачей сегментации изображений, а точнее - выделением кожи на фотографиях. 

В итоге мы получим классификатор, который по RBG представлению цвета пикселя будет пытаться определить, принадлежит пиксель к участку кожи или нет.

<img src="skin_segmentation2.png" />

Для начала скачаем выборку по следующей ссылке: https://archive.ics.uci.edu/ml/machine-learning-databases/00229/Skin_NonSkin.txt

In [ ]:
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/00229/Skin_NonSkin.txt > Skin_NonSkin.txt

## Задаем кодировку

В Python кодировка файла скрипта задается с помощью так называемого magic comment: # coding: utf-8
(либо другая кодировка по необходимости).

Волшебный комментарий должен быть в первых двух строчках скрипта, иначе он игнорируется интерпретатором. Добавив этот комментарий в скрипт можно, например, комфортно использовать в коде русскоязычные строки и комментарии. Однако для ipython notebook такой способ не подходит ведь разные блоки кода могут исполняться в разной последовательности. Есть другой способ, им и воспользуемся:

In [ ]:
# import sys
# reload(sys)
# sys.setdefaultencoding("utf-8")

После исполнения этого блока проблем с русскоязычными комментариями в коде быть не должно.

## Чтение выборки

Давайте посмотрим на выборку и попробуем понять, какой смысл имеют

In [ ]:
dataset_path = "Skin_NonSkin.txt"
dataset_file = open(dataset_path, 'r')
dataset_text = dataset_file.read().split('\n')


In [ ]:
dataset = [map(int, line.split('\t')) for line in dataset_text if len(line) > 0]
print dataset[:10]

In [ ]:
X = [line[:-1] for line in dataset]
y = [line[-1] for line in dataset]

print X[:5]

#print "123",

print y[:5]

In [ ]:
import numpy as np

np.random.seed(0)
indices = np.random.permutation(len(X))
X = np.array(X)
y = np.array(y)
test_size = 100000
X_train = X[indices[:-test_size]]
y_train = y[indices[:-test_size]]
X_test  = X[indices[-test_size:]]
y_test  = y[indices[-test_size:]]
print X.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from tqdm import trange
model = [[], []]
y_pred = [[], []]
score = [[], []]
n_neighbors = range(1,11)
weights = ['uniform', 'distance']

for i in trange(len(n_neighbors)):
    model[0].append(KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                                      metric_params=None, n_jobs=1, n_neighbors=n_neighbors[i], p=2,
                                      weights=weights[0])
                   )
    model[1].append(KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                                      metric_params=None, n_jobs=1, n_neighbors=n_neighbors[i], p=2,
                                      weights=weights[1])
                   )
    model[0][i].fit(X_train, y_train)
    model[1][i].fit(X_train, y_train)
    
    y_pred[0].append(model[0][i].predict(X_test))
    y_pred[1].append(model[1][i].predict(X_test))
    
    score[0].append(float(sum(y_test == y_pred[0][i]))/test_size)
    score[1].append(float(sum(y_test == y_pred[1][i]))/test_size)


In [ ]:
df = pd.DataFrame(score, columns=n_neighbors, index=weights)
print df

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(df.ix[0], label=weights[0])
plt.plot(df.ix[1], label=weights[1])
plt.legend()
plt.show()

## Чтение и обработка изображения

Для работы с графиками и изображениями в Python есть библиотека Matplotlib. На сайте библиотеки www.matplotlib.org описан API и есть большое количество примеров в разделе gallery. При необходимости построить график можно зайти в галерею, найти визуально картинку, наиболее похожую на нужную и посмотреть код.

Но сейчас мы воспользуемся matplotlib для работы с изображением, на котором хотим найти участки кожи. Для начала считаем изображение:

In [ ]:
import matplotlib.cbook as cbook

image_path = "nikolaev.jpg"

image_file = open(image_path, 'rb')
#image_file = cbook.get_sample_data(image_path)
image = plt.imread(image_file)

plt.imshow(image)
plt.axis('off') # clear x- and y-axes
plt.show()

Посмотрим, в каком формате хранится изображение в переменой image:

In [ ]:
print type(image)

Поскольку это массив, попробуем вывести несколько его первых элементов:

In [ ]:
print image[:3]

Похоже на массив строк пикселей. Каждая строка - массив пикселей, а пиксель - массив из трех чисел - RGB представления цвета пикселя.

Обучим наш классификатор на всей обучающей выборке и прогоним классификатор по всем пикселям изображения, заменяя пиксели кожи на ярко зелёные:

In [ ]:
X[:,[0,2]] = X[:,[2,0]]
model = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                                      metric_params=None, n_jobs=1, n_neighbors=n_neighbors[3], p=2,
                                      weights=weights[1]
                             )
model.fit(X, y)

In [ ]:
for i, line in enumerate(image):
    classes = model.predict(line) # predict принимает список объектов
    for j, ans in enumerate(classes):
        if ans == 1:
            image[i, j, 0] = image[i, j, 2] = 0
            image[i, j, 1] = 255

In [ ]:
plt.imshow(image)
plt.show()

Не правда ли, какой-то подозрительный результат?

## Задание

#### На семинаре:

1. Выяснить, почему на изображении не выделились участки кожи и исправить ситуацию. Подсказка: читайте документацию библиотек либо пробуйте визуализировать цвета, которые классификатор относит к коже. Блокнот с решением проблемы выложить на github.
1. Сравнить качество на отложенной выборке при разном количестве соседей k в kNN (от 1 до 10) и при разных весах во взвешенной версии kNN (см. документацию scikit-learn). 
1. Блокнот с исправлением детектирования кожи и экспериментами выслать на datamininginaction@gmail.com

#### Дополнительные задания (к следующему занятию):

1. С помощью matplotlib построить трёхмерную визуализацию выборки и выслать на тот же адрес.
1. Разобрать презенташку про numpy, scipy, matplotlib: http://mit.spbau.ru/files/scipy.pdf
1. Разобрать Crash Course in Python for scientists.

In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

color=X/255.

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X[:, 0], X[:, 1], X[:, 2], s=5, color=color)

ax.set_xlabel('Red')
ax.set_ylabel('Green')
ax.set_zlabel('Blue')

plt.savefig('dataset_visualization.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(7, 21))
ax1 = fig.add_subplot(311)
ax1.scatter(X[:, 0], X[:, 1], s=5, color=color)
ax1.set_xlabel('Red')
ax1.set_ylabel('Green')

ax2 = fig.add_subplot(312)
ax2.scatter(X[:, 1], X[:, 2], s=5, color=color)
ax2.set_xlabel('Green')
ax2.set_ylabel('Blue')

ax3 = fig.add_subplot(313)
ax3.scatter(X[:, 2], X[:, 0], s=5, color=color)
ax3.set_xlabel('Blue')
ax3.set_ylabel('Red')
plt.show()